In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import wordpunct_tokenize  
from nltk.corpus import stopwords

import re
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from scipy.stats import zscore


nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gradi\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\gradi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gradi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gradi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#label : 1-fake, 0-real
df = pd.read_csv('WELFake_Dataset.csv', index_col=0)
df.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


Eliminare duplicate

In [3]:
df.isnull().sum()

title    558
text      39
label      0
dtype: int64

In [5]:
df = df[df['title'].notna() & df['text'].notna()]
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [4]:
duplicate_groups = df.groupby(['title', 'text','label'])
duplicates = duplicate_groups.filter(lambda x: len(x) > 1)
print(f"Nr duplicate: {len(duplicates)}")

Nr duplicate: 16300


In [6]:
duplicates_sorted = duplicates.sort_values(by='title', ascending=False).reset_index(drop=True)
duplicates_sorted.head(10)

,title,text,label
0,“You’re Not Welcome!” Obama As Welcome At Rose...,Roseberg residents and families of victims are...,1
1,“You’re Not Welcome!” Obama As Welcome At Rose...,Roseberg residents and families of victims are...,1
2,"“You Ruined Your Own Communities, Don’t Ruin O...","- < “You Ruined Your Own Communities, Don’t Ru...",1
3,"“You Ruined Your Own Communities, Don’t Ruin O...","- < “You Ruined Your Own Communities, Don’t Ru...",1
4,“YOU’RE HIRED!” Trump Pulls Unemployed Vet Fro...,"No matter which candidate you support, this mo...",1
5,“YOU’RE HIRED!” Trump Pulls Unemployed Vet Fro...,"No matter which candidate you support, this mo...",1
6,“Would You Rather Do A Job You Hate And Not Pa...,"Filmmaker and patriot, Dennis Michael Lynch is...",1
7,“Would You Rather Do A Job You Hate And Not Pa...,"Filmmaker and patriot, Dennis Michael Lynch is...",1
8,"“Wikileaks is the Mossad, Stupid, Not the Russ...",Russian experts collecting evidence of anti-go...,1
9,"“Wikileaks is the Mossad, Stupid, Not the Russ...",Russian experts collecting evidence of anti-go...,1


In [7]:
df = df.drop_duplicates(subset=['title', 'text'], keep='first').reset_index(drop=True)

Eliminare spatii multiple

In [8]:
df['title'] = df['title'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

Pentru inceput voi 'sparge' textul intr-o lista de stringuri ca sa analizez dimensionalitatea stirilor.

In [9]:
#voi folosi wordpunct_tokenize deoarece word_tokenize 
#nu recunoaste punctuatia, iar in texte mai exista greseli de scriere(de exemplu lipsa spatiului dupa punctul de la finalul unei propozitii)

df['text'] = df['text'].apply(lambda x: str(x) if pd.notna(x) else "") #transformam in string daca e nan sau alt tip de date
df['tokens'] = df['text'].apply(lambda x: [token for token in wordpunct_tokenize(x) if token.strip() != ""])  #validare sa nu am tokenuri goale

df['title'] = df['title'].apply(lambda x: str(x) if pd.notna(x) else "")
df['title_tokens'] = df['title'].apply(lambda x: [token for token in wordpunct_tokenize(x) if token.strip() != ""])

df['tokens_count'] = df['tokens'].apply(lambda x: len(x))
df['title_tokens_count'] = df['title_tokens'].apply(lambda x: len(x))

In [10]:
print((df['tokens_count'] == 0).sum())
print((df['title_tokens_count'] == 0).sum())

529
0


529 stiri goale, le elimin

In [11]:
df = df[df['tokens_count'] != 0]
(df['tokens_count'] == 0).sum()

np.int64(0)

In [14]:
def clean_tokens(tokens):
    clean = []
    for token in tokens:
        token = token.lower()
        token = re.sub(r"[^\w\d]", "", token)  # litere si cifre
        if len(token) > 2:
            clean.append(token)
    return clean

df['clean_text'] = df['tokens'].apply(clean_tokens)
df['clean_title'] = df['title_tokens'].apply(clean_tokens)

In [15]:
df['words_count'] = df['clean_text'].apply(lambda x: len(x))
df['title_words_count'] = df['clean_title'].apply(lambda x: len(x))

In [21]:
print((df['words_count'] == 0).sum())
print((df['title_words_count'] == 0).sum())

1
1


In [25]:
df = df[(df['words_count'] != 0) & (df['title_words_count'] != 0)]

print((df['words_count'] == 0).sum())
print((df['title_words_count'] == 0).sum())

0
0


Eliminare stopwords

In [19]:
stop_words = stopwords.words('english')
df['clean_text_without_stopwords'] = df['clean_text'].apply(lambda x: [word for word in x if word not in stop_words and len(word) > 2])
df['clean_title_without_stopwords'] = df['clean_title'].apply(lambda x: [word for word in x if word not in stop_words and len(word) > 2])

In [20]:
df['words_count_without_stopwords'] = df['clean_text_without_stopwords'].apply(lambda x: len(x))
df['title_words_count_without_stopwords'] = df['clean_text_without_stopwords'].apply(lambda x: len(x))

In [23]:
print((df['words_count_without_stopwords'] == 0).sum())
print((df['title_words_count_without_stopwords'] == 0).sum())

2
2


In [26]:
df = df[(df['words_count_without_stopwords'] != 0) & (df['title_words_count_without_stopwords'] != 0)]

print((df['words_count_without_stopwords'] == 0).sum())
print((df['title_words_count_without_stopwords'] == 0).sum())

0
0


Lemmatizare

In [28]:
lemmatizer = WordNetLemmatizer()

def lemmatize_with_pos(tokens):
    pos_tags = pos_tag(tokens)
    
    lemmatized_tokens = []
    for word, tag in pos_tags:
        if tag.startswith('NN'):
            pos = 'n'  # substantiv
        elif tag.startswith('VB'):
            pos = 'v'  # verb
        elif tag.startswith('JJ'):
            pos = 'a'  # adjectiv
        else:
            pos = 'n'  # substantiv (valoare implicita)
        lemmatized_tokens.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_tokens

In [ ]:
df['text_lemmatized'] = df['clean_text_without_stopwords'].apply(lemmatize_with_pos)
df['title_lemmatized'] = df['clean_title_without_stopwords'].apply(lemmatize_with_pos)

In [33]:
df.iloc[234]

title                                  Re: WE’RE NOT NASTY! Like all Democrats, Sally...
text                                   WE’RE NOT NASTY! Why does Sally Kohn hate wome...
label                                                                                  1
tokens                                 [WE, ’, RE, NOT, NASTY, !, Why, does, Sally, K...
title_tokens                           [Re, :, WE, ’, RE, NOT, NASTY, !, Like, all, D...
tokens_count                                                                         319
title_tokens_count                                                                    24
clean_text                             [not, nasty, why, does, sally, kohn, hate, wom...
clean_title                            [not, nasty, like, all, democrats, sally, kohn...
words_count                                                                          212
title_words_count                                                                     16
clean_text_without_st

In [34]:
df.iloc[32456]

title                                  Papua New Guinea starts dismantling detention ...
text                                   SYDNEY (Reuters) - Papua New Guinea began dism...
label                                                                                  0
tokens                                 [SYDNEY, (, Reuters, ), -, Papua, New, Guinea,...
title_tokens                           [Papua, New, Guinea, starts, dismantling, dete...
tokens_count                                                                         613
title_tokens_count                                                                    14
clean_text                             [sydney, reuters, papua, new, guinea, began, d...
clean_title                            [papua, new, guinea, starts, dismantling, dete...
words_count                                                                          457
title_words_count                                                                     12
clean_text_without_st

In [36]:
cols = ['title', 'text', 'label', 'clean_text', 'clean_title', 'clean_text_without_stopwords', 'clean_title_without_stopwords', 'text_lemmatized', 'title_lemmatized']

In [37]:
df[cols].head()

,title,text,label,clean_text,clean_title,clean_text_without_stopwords,clean_title_without_stopwords,text_lemmatized,title_lemmatized
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,"[comment, expected, from, barack, obama, membe...","[law, enforcement, high, alert, following, thr...","[comment, expected, barack, obama, members, fy...","[law, enforcement, high, alert, following, thr...","[comment, expect, barack, obama, member, fyf91...","[law, enforcement, high, alert, follow, threat..."
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last n...",1,"[now, most, the, demonstrators, gathered, last...","[unbelievable, obama, attorney, general, says,...","[demonstrators, gathered, last, night, exercis...","[unbelievable, obama, attorney, general, says,...","[demonstrator, gather, last, night, exercise, ...","[unbelievable, obama, attorney, general, say, ..."
2,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"[dozen, politically, active, pastors, came, he...","[bobby, jindal, raised, hindu, uses, story, ch...","[dozen, politically, active, pastors, came, pr...","[bobby, jindal, raised, hindu, uses, story, ch...","[dozen, politically, active, pastor, come, pri...","[bobby, jindal, raise, hindu, us, story, chris..."
3,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"[the, sarmat, missile, dubbed, satan, will, re...","[satan, russia, unvelis, image, its, terrifyin...","[sarmat, missile, dubbed, satan, replace, flie...","[satan, russia, unvelis, image, terrifying, ne...","[sarmat, missile, dub, satan, replace, fly, mi...","[satan, russia, unvelis, image, terrify, new, ..."
4,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,"[all, can, say, this, one, about, time, someon...","[about, time, christian, group, sues, amazon, ...","[say, one, time, someone, sued, southern, pove...","[time, christian, group, sues, amazon, splc, d...","[say, one, time, someone, sue, southern, pover...","[time, christian, group, sue, amazon, splc, de..."


In [ ]:
df['z_score'] = zscore(df['words_count'])
print((df['z_score'] >=3).sum())
print((df['z_score'] < -3).sum())

819
0


In [ ]:
df = df[df['z_score'] < 3]
print((df['z_score'] >=3).sum())

0


In [44]:
df[cols].to_csv('preprocessed_data.csv', index=False)

Pentru fiecare coloana contine siruri de strings, se va folosi .apply(lambda x: eval(x))